In [1]:
using CSV
using DataFrames


In [ ]:

function load_from_clipboard()
    # Get clipboard content based on the operating system
    content = ""

    if Sys.iswindows()
        content = read(`powershell Get-Clipboard`, String)
    elseif Sys.isapple()
        content = read(`pbpaste`, String)
    elseif Sys.isunix()
        content = read(`xclip -o -selection clipboard`, String)
    else
        error("Unsupported OS")
    end

    # Strip whitespace
    content = strip(content)

    # Check if the content resembles tabular data (Excel output)
    if occursin(r"[\t]", content)  # Check for tab characters
        # Treat as CSV with tab delimiter
        return CSV.read(IOBuffer(content), DataFrame; delim='\t')
    
    elseif occursin(r"^\s*[\d,.]+\s*$", content)  # Check if it's a single number
        return parse(Float64, content)

    elseif occursin(r"^[^\n,]*([,][^\n]*)*$", content)  # Check for simple CSV format
        return CSV.read(IOBuffer(content), DataFrame)

    elseif occursin(r"\n", content)  # Check for multi-line text
        return split(content, '\n')

    else
        return content  # Default case for plain text
    end
end



In [ ]:

# Usage example:
data = load_from_clipboard()
println(data)


In [ ]:
using CSV
using DataFrames

function load_from_clipboard()
    # Get clipboard content based on the operating system
    content = ""

    if Sys.iswindows()
        content = read(`powershell Get-Clipboard`, String)
    elseif Sys.isapple()
        content = read(`pbpaste`, String)
    elseif Sys.isunix()
        content = read(`xclip -o -selection clipboard`, String)
    else
        error("Unsupported OS")
    end

    # Strip whitespace
    content = strip(content)

    # Try to classify the clipboard content
    if isempty(content)
        return nothing  # Empty clipboard
    end

    # Detect tabular data (with tabs or commas, multi-line)
    if occursin(r"[\t]", content) || occursin(r"\n.*[,]", content)
        try
            return CSV.read(IOBuffer(content), DataFrame; delim='\t')
        catch
            try
                return CSV.read(IOBuffer(content), DataFrame)
            catch
                return split(content, '\n')  # Fallback for tabular-like data
            end
        end

    # Detect single numeric value
    elseif occursin(r"^\s*[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?\s*$", content)
        return parse(Float64, content)

    # Detect list-like data (comma-separated single line)
    elseif occursin(r"^([^,\n]+,)+[^,\n]+$", content)
        return split(content, ",")

    # Detect multi-line text
    elseif occursin(r"\n", content)
        return split(content, '\n')

    # Fallback to plain text
    else
        return content
    end
end


In [ ]:

# Test function
function test_load_from_clipboard()
    results = Dict()

    # Test case 1: Single numeric value
    input1 = "42.5"
    results[:test_numeric] = load_from_clipboard_simulated(input1) == 42.5

    # Test case 2: Tab-delimited data
    input2 = "Name\tAge\tCity\nAlice\t30\tNew York\nBob\t25\tLos Angeles"
    expected_df2 = DataFrame(Name=["Alice", "Bob"], Age=[30, 25], City=["New York", "Los Angeles"])
    results[:test_tabular] = isequal(load_from_clipboard_simulated(input2), expected_df2)

    # Test case 3: Simple CSV data
    input3 = "Name,Age,City\nAlice,30,New York\nBob,25,Los Angeles"
    expected_df3 = DataFrame(Name=["Alice", "Bob"], Age=[30, 25], City=["New York", "Los Angeles"])
    results[:test_csv] = isequal(load_from_clipboard_simulated(input3), expected_df3)

    # Test case 4: Comma-separated values
    input4 = "apple,banana,cherry"
    results[:test_list] = load_from_clipboard_simulated(input4) == ["apple", "banana", "cherry"]

    # Test case 5: Multi-line text
    input5 = "Line 1\nLine 2\nLine 3"
    results[:test_multiline] = load_from_clipboard_simulated(input5) == ["Line 1", "Line 2", "Line 3"]

    # Test case 6: Plain text
    input6 = "Hello, World!"
    results[:test_plaintext] = load_from_clipboard_simulated(input6) == "Hello, World!"

    # Return results
    return results
end

# Helper function to simulate clipboard content for testing
function load_from_clipboard_simulated(mock_content::String)
    # Simulates the content as if it's from the clipboard
    return eval(Meta.parse(replace(string(load_from_clipboard()), "read(" => "mock_content")))
end



In [ ]:
test_load_from_clipboard()